# Actividad 2 Analisis de Informacion No Estructurada

# - Javier Gilabert Sabater

1. LeaelcontenidodelficherocsvenunDataFrame.
2. Realiceelpre-procesamientoqueconsiderenecesario.Puede
utilizar funciones de la librería NLTK o spaCy, a su voluntad. Recomendamos una escritura modular del código, para poder hacer pruebas posteriormente, y contestar a las preguntas del punto 6.
3. ConviertaelcorpusdedocumentosenunamatrizTF-idf.
4. Dividaenunsubconjuntodeentrenamientoyotrodeevaluación.
5. Llegadosaestepunto,realicemodelosdeentrenamientoalmenos
con algoritmos de clasificador bayesiano ingenuo, máquinas SVM y un modelo basado en árbol de decisión. Obtenga resultados de accuracy de la clasificación, así como las matrices de confusión para los tres modelos.
6. Contestealassiguientespreguntasbasándoteenevidenciasde código. ¿Tiene influencia en el resultado final el número máximo de features a utilizar? Prueba al menos dos números de features diferentes para los tres algoritmos y mide los resultados. ¿Modifica el resultado si no se eliminan las stop words? Pruébalo para los tres algoritmos y mide los resultados.
7. Imagínesequeesteentregableesunalaborquelehansolicitadoen un entorno profesional, y que tiene que entregar esta documentación para comentar lo que ha descubierto (datos de entrada, rendimiento de los modelos, o cualquier descubrimiento que pueda ser importante). Comente los resultados obtenidos.


## 1. Lea el contenido del fichero csv en un DataFrame.

In [1]:
import pandas as pd

In [ ]:
df = pd.read_csv("spam.csv")